# Activity — Estimate Intrinsic and Extrinsic Parameters

Do all imports.

In [ ]:
# For numerical methods
import numpy as np
from scipy.spatial.transform import Rotation

# For comparison
import cv2

Create random number generator with a particular seed so we can reproduce results.

In [ ]:
rng = np.random.default_rng(20)

Define a function to print things nicely.

In [ ]:
def myprint(M):
    if M.shape:
        with np.printoptions(linewidth=150, formatter={'float': lambda x: f'{x:10.4f}'}):
            print(M)
    else:
        print(f'{M:10.4f}')

#### Create dataset

Choose intrinsic parameters.

In [ ]:
K_true = np.array([
    [150., 0., 100.],
    [0., 160., 75.],
    [0., 0., 1.],
])

Choose extrinsic parameters (these are poses **of the camera in the world frame**).

In [ ]:
poses_true = []

R = np.eye(3)
p = np.array([0., 0., 1.])
poses_true.append(
    np.row_stack([
        np.column_stack([R.T, -R.T @ p]),
        np.array([0., 0., 0., 1.]),
    ])
)

R = Rotation.from_rotvec((np.pi / 6) * np.array([1., 0., 0.])).as_matrix()
p = np.array([0.1, 0., 0.9])
poses_true.append(
    np.row_stack([
        np.column_stack([R.T, -R.T @ p]),
        np.array([0., 0., 0., 1.]),
    ])
)

R = Rotation.from_rotvec((np.pi / 6) * np.array([0., 1., 0.])).as_matrix()
p = np.array([0., 0.1, 1.1])
poses_true.append(
    np.row_stack([
        np.column_stack([R.T, -R.T @ p]),
        np.array([0., 0., 0., 1.]),
    ])
)

R = Rotation.from_rotvec((np.pi / 3) * np.array([0., 0., 1.])).as_matrix()
p = np.array([-0.1, 0.1, 1.])
poses_true.append(
    np.row_stack([
        np.column_stack([R.T, -R.T @ p]),
        np.array([0., 0., 0., 1.]),
    ])
)

Compute homographies (scaled uniformly at random).

In [ ]:
homographies = []
for pose in poses_true:
    R_inC_ofW = pose[0:3, 0:3].T
    p_inC_ofW = - pose[0:3, 0:3].T @ pose[0:3, 3]
    x_inC_ofW = R_inC_ofW[:, 0]
    y_inC_ofW = R_inC_ofW[:, 1]
    homographies.append(rng.uniform(low=0.1, high=10.) * K_true @ np.column_stack([x_inC_ofW, y_inC_ofW, p_inC_ofW]))
homographies = np.array(homographies)

#### Estimate intrinsic parameters

Suppose

$$b = \begin{bmatrix} b_1 \\ \vdots \\ b_6 \end{bmatrix}$$

contains the unique elements of the symmetric matrix

$$B = \begin{bmatrix} b_1 & b_4 & b_6 \\ b_4 & b_2 & b_5 \\ b_6 & b_5 & b_3 \end{bmatrix}.$$

Given $a, c \in \mathbb{R}^3$, define a function that returns the vector $m \in \mathbb{R}^3$ for which

$$a^T B c = m^T b.$$

In [ ]:
# FIXME
def get_m(a, c):
    assert(a.shape == (3,))
    assert(c.shape == (3,))
    return np.zeros(6)

Define a function that estimates the camera matrix $K$ from a list of homographies $H_1, \dotsc, H_n$.

In [ ]:
# FIXME
def get_intrinsic_parameters(homographies):
    return np.eye(3)

Apply this function to find the intrinsic parameters.

In [ ]:
K = get_intrinsic_parameters(homographies)

myprint(K)
assert(np.allclose(K, K_true))

#### Estimate extrinsic parameters

Define a function that implements the wedge operator.

In [ ]:
def skew(v):
    assert(type(v) == np.ndarray)
    assert(v.shape == (3,))
    return np.array([[0., -v[2], v[1]],
                     [v[2], 0., -v[0]],
                     [-v[1], v[0], 0.]])

Define a function that estimates the pose $R_C^W$ that corresponds to each homography.

In [ ]:
# FIXME
def get_extrinsic_parameters(homographies, K):
    poses = []
    for H in homographies:
        poses.append(np.array([
            [1., 0., 0., 0.],
            [0., 1., 0., 0.],
            [0., 0., 1., 0.],
            [0., 0., 0., 1.],
        ]))
    
    # Return all poses
    return np.array(poses)

Apply this function to find extrinsic parameters.

In [ ]:
poses = get_extrinsic_parameters(homographies, K)

for pose, pose_true in zip(poses, poses_true):
    assert(np.allclose(pose, pose_true))